In [1]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torch.nn.functional as F
import numpy as np

learning_rate = 1e-4
keep_prob_rate = 0.7
max_epoch = 3
BATCH_SIZE = 50

DOWNLOAD_MNIST = False
if not(os.path.exists('./mnist/')) or not os.listdir('./mnist/'):
    # not mnist dir or mnist is empyt dir
    DOWNLOAD_MNIST = True

train_data = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)
test_x = Variable(torch.unsqueeze(test_data.test_data, dim=1), volatile=True).type(torch.FloatTensor)[:500]/255.
test_y = test_data.test_labels[:500].numpy()

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,  # MNIST images are grayscale, so only 1 channel
                out_channels=32,  # Number of output channels
                kernel_size=7,  # 7x7 kernel
                stride=1,  # Stride of 1
                padding=3,  # Padding to keep the same size (same padding)
            ),
            nn.ReLU(),  # Activation function
            nn.MaxPool2d(2),  # 2x2 pooling
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,  # Input channels from the previous layer
                out_channels=64,  # Number of output channels
                kernel_size=5,  # 5x5 kernel
                stride=1,  # Stride of 1
                padding=2,  # Padding to keep the same size (same padding)
            ),
            nn.ReLU(),  # Activation function
            nn.MaxPool2d(2),  # 2x2 pooling
        )
        self.out1 = nn.Linear(7*7*64, 1024, bias=True)  # Fully connected layer
        self.dropout = nn.Dropout(keep_prob_rate)
        self.out2 = nn.Linear(1024, 10, bias=True)  # Output layer

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)  # Flatten the output for the fully connected layer
        out1 = self.out1(x)
        out1 = F.relu(out1)
        out1 = self.dropout(out1)
        out2 = self.out2(out1)
        output = F.softmax(out2, dim=1)  # Softmax for classification
        return output

def test(cnn):
    global prediction
    y_pre = cnn(test_x)
    _, pre_index = torch.max(y_pre, 1)
    pre_index = pre_index.view(-1)
    prediction = pre_index.data.numpy()
    correct = np.sum(prediction == test_y)
    return correct / 500.0

def train(cnn):
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(max_epoch):
        for step, (x_, y_) in enumerate(train_loader):
            x, y = Variable(x_), Variable(y_)
            output = cnn(x)
            loss = loss_func(output, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if step != 0 and step % 20 == 0:
                print("=" * 10, step, "=" * 5, "=" * 5, "test accuracy is", test(cnn), "=" * 10)

if __name__ == '__main__':
    cnn = CNN()
    train(cnn)

100%|██████████| 9.91M/9.91M [00:00<00:00, 34.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.19MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 10.5MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.59MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
<ipython-input-1-dddd7fb1d9d9>:24: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  test_x = Variable(torch.unsqueeze(test_data.test_data, dim=1), volatile=True).type(torch.FloatTensor)[:500]/255.
/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


========== 20 ===== ===== test accuracy is 0.25 ==========
========== 40 ===== ===== test accuracy is 0.27 ==========
========== 60 ===== ===== test accuracy is 0.412 ==========
========== 80 ===== ===== test accuracy is 0.61 ==========
========== 100 ===== ===== test accuracy is 0.596 ==========
========== 120 ===== ===== test accuracy is 0.662 ==========
========== 140 ===== ===== test accuracy is 0.68 ==========
========== 160 ===== ===== test accuracy is 0.712 ==========
========== 180 ===== ===== test accuracy is 0.762 ==========
========== 200 ===== ===== test accuracy is 0.78 ==========
========== 220 ===== ===== test accuracy is 0.814 ==========
========== 240 ===== ===== test accuracy is 0.842 ==========
========== 260 ===== ===== test accuracy is 0.84 ==========
========== 280 ===== ===== test accuracy is 0.836 ==========
========== 300 ===== ===== test accuracy is 0.856 ==========
========== 320 ===== ===== test accuracy is 0.872 ==========
========== 340 ===== ===== test ac